# Teste das Funções da Camada Bronze

Este notebook tem como objetivo testar e validar todas as funções da camada bronze do pipeline de dados financeiros.

## 1. Importação de Bibliotecas e Módulos

Vamos importar todos os módulos necessários para testar as funções da camada bronze.

In [ ]:
import json
from datetime import datetime
import pandas as pd
from pyspark.sql import types as T

# Importa as funções personalizadas
from utilitarios.configuracoes import (
    obter_configuracao,
    obter_lista_configuracoes,
    obter_nome_tabela,
    obter_metadados_tabela,
    timestamp_ingestao,
)
from utilitarios.fontes_dados import (
    buscar_historico_b3,
    buscar_series_bacen,
    criar_dataframe_vazio,
    spark,
)

## 2. Configurar Ambiente de Teste

Vamos definir algumas variáveis de teste e configurar o ambiente para os testes.

In [ ]:
# Configurações de teste para B3
tickers_teste = ["PETR4", "VALE3", "ITUB4"]  # Conjunto reduzido para teste
data_inicial = "01/01/2024"
data_final = datetime.utcnow().strftime("%d/%m/%Y")

# Configurações de teste para BACEN
series_teste = {
    "selic": 1178,  # Taxa Selic diária
    "ipca": 433,    # IPCA mensal
}

# Schema para validação de cotações B3
schema_cotacoes = T.StructType([
    T.StructField("Date", T.TimestampType(), False),
    T.StructField("Open", T.DoubleType(), True),
    T.StructField("High", T.DoubleType(), True),
    T.StructField("Low", T.DoubleType(), True),
    T.StructField("Close", T.DoubleType(), True),
    T.StructField("Volume", T.DoubleType(), True),
    T.StructField("Dividends", T.DoubleType(), True),
    T.StructField("Stock_Splits", T.DoubleType(), True),
    T.StructField("ticker", T.StringType(), False),
    T.StructField("ingestion_timestamp", T.TimestampType(), False),
])

print(f"Configurações de teste definidas:")
print(f"- Tickers B3: {tickers_teste}")
print(f"- Período: {data_inicial} até {data_final}")
print(f"- Séries BACEN: {list(series_teste.keys())}")

## 3. Testar Função bronze_cotacoes_b3

Vamos testar a função de coleta de cotações da B3 em diferentes cenários.

In [ ]:
# Teste 1: Buscar dados históricos
print("Teste 1: Buscando dados históricos da B3...")
df_cotacoes = buscar_historico_b3(tickers_teste, data_inicial, data_final)
print("\nEstrutura dos dados retornados:")
print(df_cotacoes.info())
print("\nPrimeiras linhas dos dados:")
print(df_cotacoes.head())

In [ ]:
# Teste 2: Converter para DataFrame Spark e validar schema
print("Teste 2: Convertendo para DataFrame Spark...")

# Adiciona timestamp de ingestão
df_cotacoes["ingestion_timestamp"] = timestamp_ingestao()

# Converte para DataFrame Spark
spark_df = spark.createDataFrame(df_cotacoes, schema=schema_cotacoes)
print("\nSchema do DataFrame Spark:")
spark_df.printSchema()
print("\nContagem de registros por ticker:")
spark_df.groupBy("ticker").count().show()

## 4. Testar Função bronze_series_bacen

Vamos testar a função de coleta de séries econômicas do BACEN.

In [ ]:
# Teste 3: Buscar séries do BACEN
print("Teste 3: Buscando séries do BACEN...")
df_series = buscar_series_bacen(series_teste, data_inicial, data_final)
print("\nEstrutura dos dados retornados:")
print(df_series.info())
print("\nPrimeiras linhas dos dados por série:")
for serie in df_series['serie'].unique():
    print(f"\nSérie: {serie}")
    print(df_series[df_series['serie'] == serie].head())

## 5. Analisar Schemas e Estruturas

Vamos examinar em detalhes os schemas e estruturas dos dados coletados.

In [ ]:
# Análise dos dados da B3
print("Análise dos dados da B3:")
print("\nTipos de dados:")
print(df_cotacoes.dtypes)
print("\nEstatísticas descritivas:")
print(df_cotacoes.describe())
print("\nVerificação de valores nulos:")
print(df_cotacoes.isnull().sum())

# Análise dos dados do BACEN
print("\nAnálise dos dados do BACEN:")
print("\nTipos de dados:")
print(df_series.dtypes)
print("\nEstatísticas descritivas por série:")
for serie in df_series['serie'].unique():
    print(f"\nSérie: {serie}")
    print(df_series[df_series['serie'] == serie]['valor'].describe())

## 6. Simular Cenários de Erro

Vamos testar diferentes cenários de erro para validar o tratamento de exceções.

In [ ]:
try:
    print("Teste de erro 1: Ticker inválido")
    df_erro = buscar_historico_b3(["TICKER_INVALIDO"], data_inicial, data_final)
    print("Resultado:", "DataFrame vazio" if df_erro.empty else "Com dados")
except Exception as e:
    print("Erro capturado:", str(e))

try:
    print("\nTeste de erro 2: Data inválida")
    df_erro = buscar_historico_b3(tickers_teste, "data_invalida", data_final)
    print("Resultado:", "DataFrame vazio" if df_erro.empty else "Com dados")
except Exception as e:
    print("Erro capturado:", str(e))

try:
    print("\nTeste de erro 3: Série BACEN inválida")
    df_erro = buscar_series_bacen({"serie_invalida": 99999}, data_inicial, data_final)
    print("Resultado:", "DataFrame vazio" if df_erro.empty else "Com dados")
except Exception as e:
    print("Erro capturado:", str(e))

: 